In [1]:
#Set Variables
CLIENT_ID = 'K5VJWGLQSG1OKPJVHU3LQEIILGRNXSNHNVSL5RTSDEQMHFTE' # your Foursquare ID
CLIENT_SECRET = 'QDEK2IBMI1OAFWHUE1TYIWQHH4F11G4SUQBVG0B0IUFLVKTO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [2]:
#Import Librairies
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
import numpy as np
import random as random
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 
!pip install folium
import folium

In [3]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
def GetVenuesNearByInDataFrame(Lat_param,Lng_param,Nb_max_venues,radius):
    url ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(
    CLIENT_ID,CLIENT_SECRET, VERSION, Lat_param, Lng_param ,Nb_max_venues ,radius)
    results = requests.get(url).json()
    print(results)
    venues = results['response']['groups'][0]['items'] 
    nearby_venues = pd.json_normalize(venues) # flatten JSON
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.city','venue.location.postalCode']
    nearby_venues =nearby_venues.loc[:, filtered_columns]
    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    nearby_venues.head()
    
    return (nearby_venues )

In [35]:
MontmartreLat=48.8857636427167
MontmartreLng=2.3349730916262725
DataFromParis = GetVenuesNearByInDataFrame(MontmartreLat,MontmartreLng,50,500)

{'meta': {'code': 200, 'requestId': '5fa57753812cc46ce9e3aeda'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Clignancourt', 'headerFullLocation': 'Clignancourt, Paris', 'headerLocationGranularity': 'neighborhood', 'totalResults': 168, 'suggestedBounds': {'ne': {'lat': 48.8902636472167, 'lng': 2.341803777256891}, 'sw': {'lat': 48.881263638216694, 'lng': 2.328142405995654}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4d541304d4a6721e9b5691ac', 'name': 'Boulangerie Alexine', 'location': {'address': '40 rue Lepic', 'lat': 48.88614133131494, 'lng': 2.3344767093658447, 'labeledLatLngs': [{'label': 'display', 'lat': 48.88614133131494, 'lng': 2.3344767093658447}], 'distance': 55, 'postalCode': '75018', 'cc': 'FR', 'city': 'Par

In [36]:
DataFromParis.shape

(50, 6)

In [37]:
SydneyLat= -33.87285279645642,
SydneyLng=151.2065676235312
DataFromSydney = GetVenuesNearByInDataFrame(SydneyLat,SydneyLng,300,2000)

{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'll must be of the form XX.XX,YY.YY (received (-33.87285279645642,),151.2065676235312)', 'requestId': '5fa5775956c1c62a0954e80b'}, 'response': {}}


KeyError: 'groups'

In [38]:
DataFromSydney

,name,categories,lat,lng,city,postalCode
0,Essence Coffee,Café,-33.831903,151.222350,Neutral Bay,2089
1,Ichibandori,Japanese Restaurant,-33.830366,151.219320,Neutral Bay,2089
2,Sushi Studio,Japanese Restaurant,-33.830263,151.218932,Neutral Bay,2089
3,Jugemu & Shimbashi,Japanese Restaurant,-33.830956,151.224333,Neutral Bay,2089
4,Sam Thai,Thai Restaurant,-33.831275,151.221663,Neutral Bay,2089
...,...,...,...,...,...,...
95,Crow Bar,Cocktail Bar,-33.826670,151.201584,Crows Nest,2065
96,finefish,Seafood Restaurant,-33.830963,151.222258,Neutral Bay,2089
97,Maggio's,Café,-33.821981,151.210529,Cammeray,2062
98,The Roots Espresso,Café,-33.839532,151.207678,North Sydney,2060


In [39]:
df_venues=pd.concat ([DataFromParis,DataFromSydney]).reset_index()
df_venues.drop(['index'],axis=1,inplace =True)
df_venues

,name,categories,lat,lng,city,postalCode
0,Boulangerie Alexine,Bakery,48.886141,2.334477,Paris,75018
1,Guilo Guilo,Japanese Restaurant,48.885942,2.337048,Paris,75018
2,Terrass'' Hotel,Hotel,48.886532,2.333159,Paris,75018
3,Al Caratello,Italian Restaurant,48.885248,2.336002,Paris,75018
4,Le Grenier à Pain,Bakery,48.885283,2.336718,Paris,75018
...,...,...,...,...,...,...
145,Crow Bar,Cocktail Bar,-33.826670,151.201584,Crows Nest,2065
146,finefish,Seafood Restaurant,-33.830963,151.222258,Neutral Bay,2089
147,Maggio's,Café,-33.821981,151.210529,Cammeray,2062
148,The Roots Espresso,Café,-33.839532,151.207678,North Sydney,2060


In [40]:
#turn the categorical data to a numeric (Hot encoding)
Data_hot = pd.get_dummies (df_venues[['categories']],prefix='',prefix_sep='')
Data_hot

# add neighborhood column back to dataframe
Data_hot['name'] = df_venues['name'] 
Data_hot['lat'] = df_venues['lat'] 
Data_hot['lng'] = df_venues['lng'] 
Data_hot['city'] = df_venues['city'] 
Data_hot['postalCode'] = df_venues['postalCode'] 


#move the fixed columns to the font.
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
fixed_columns = [Data_hot.columns[-1]] + list(Data_hot.columns[:-1])
Data_hot = Data_hot[fixed_columns]
Data_hot

,name,lat,lng,city,postalCode,American Restaurant,Bakery,Bar,Beach,Beer Bar,...,Scenic Lookout,Seafood Restaurant,South Indian Restaurant,Street Art,Tennis Court,Thai Restaurant,Theater,Trail,Wine Bar,Wine Shop
0,Boulangerie Alexine,48.886141,2.334477,Paris,75018,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guilo Guilo,48.885942,2.337048,Paris,75018,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Terrass'' Hotel,48.886532,2.333159,Paris,75018,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Al Caratello,48.885248,2.336002,Paris,75018,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Le Grenier à Pain,48.885283,2.336718,Paris,75018,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Crow Bar,-33.826670,151.201584,Crows Nest,2065,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146,finefish,-33.830963,151.222258,Neutral Bay,2089,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
147,Maggio's,-33.821981,151.210529,Cammeray,2062,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
148,The Roots Espresso,-33.839532,151.207678,North Sydney,2060,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
city_grouped = Data_hot.groupby(['postalCode']).mean().reset_index()
city_grouped

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,...,Scenic Lookout,Seafood Restaurant,South Indian Restaurant,Street Art,Tennis Court,Thai Restaurant,Theater,Trail,Wine Bar,Wine Shop
0,2055,-33.834709,151.206667,0.000000,0.500000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,2060,-33.837990,151.208102,0.038462,0.000000,0.115385,0.00,0.0,0.038462,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,2061,-33.844796,151.212978,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,2062,-33.822669,151.211810,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000
4,2065,-33.826548,151.201621,0.000000,0.000000,0.058824,0.00,0.0,0.000000,0.058824,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.00,0.000000,0.000000
5,2088,-33.830200,151.233926,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
6,2089,-33.831717,151.221577,0.000000,0.041667,0.041667,0.00,0.0,0.000000,0.000000,...,0.041667,0.041667,0.000000,0.000000,0.041667,0.041667,0.000000,0.00,0.000000,0.000000
7,2090,-33.829859,151.227430,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000
8,75009,48.883429,2.332437,0.000000,0.000000,0.000000,0.00,0.5,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
9,75018,48.885830,2.335438,0.000000,0.083333,0.083333,0.00,0.0,0.062500,0.000000,...,0.000000,0.020833,0.000000,0.020833,0.000000,0.000000,0.020833,0.00,0.020833,0.020833


In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['postalCode'] = city_grouped['postalCode']

for ind in np.arange(city_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,postalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2055,lng,Bakery,Café,Wine Shop,Churrascaria,Gym,Garden,Fruit & Vegetable Store,French Restaurant,Electronics Store
1,2060,lng,Café,Bar,Japanese Restaurant,Coffee Shop,American Restaurant,Park,Electronics Store,Juice Bar,Bistro
2,2061,lng,Indian Restaurant,Park,Wine Shop,Churrascaria,Garden,Fruit & Vegetable Store,French Restaurant,Electronics Store,Dessert Shop
3,2062,lng,Thai Restaurant,Fruit & Vegetable Store,Café,Butcher,Japanese Restaurant,Wine Shop,Costume Shop,Coffee Shop,Comedy Club
4,2065,lng,Japanese Restaurant,Café,Cocktail Bar,Thai Restaurant,Bar,South Indian Restaurant,Indian Chinese Restaurant,Churrascaria,Bookstore


#Assign randomly a cluster to each datapoints
#k is the number of clusters
k = 5
df = pd.DataFrame(np.random.randint(0,k,size=(city_grouped.shape[0], 1)), columns=list('A'))
city_grouped['cluster'] = df
fixed_columns = [city_grouped.columns[-1]] + list(city_grouped.columns[:-1])
city_grouped = city_grouped[fixed_columns]
city_grouped
#city_grouped.head(20)

Apply K-means to cluster the Areas according their most common venues

In [44]:
X = city_grouped.values[:,4:]
X

array([[0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0],
       [0.0, 0.11538461538461539, 0.0, 0.0, 0.038461538461538464, 0.0,
        0.0, 0.0, 0.46153846153846156, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.038461538461538464, 0.0, 0.0, 0.0, 0.0, 0.038461538461538464,
        0.0, 0.0, 0.038461538461538464, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.07692307692307693, 0.038461538461538464, 0.0,
        0.038461538461538464, 0.0, 0.0, 0.038461538461538464, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.038461538461538464, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0

In [120]:
k=4
k_means = KMeans(init="k-means++", n_clusters=k, n_init=21)
k_means.fit(X)
labels = k_means.labels_
k_means

KMeans(n_clusters=4, n_init=21)

In [121]:
labels
city_grouped["Label"] = labels

In [122]:
city_grouped

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,...,South Indian Restaurant,Street Art,Tennis Court,Thai Restaurant,Theater,Trail,Wine Bar,Wine Shop,Label,color
0,2055,-33.834709,151.206667,0.000000,0.500000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,3,#000000
1,2060,-33.837990,151.208102,0.038462,0.000000,0.115385,0.00,0.0,0.038462,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1,#FF6F61
2,2061,-33.844796,151.212978,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0,#ff8000
3,2062,-33.822669,151.211810,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,1,#FF6F61
4,2065,-33.826548,151.201621,0.000000,0.000000,0.058824,0.00,0.0,0.000000,0.058824,...,0.058824,0.000000,0.000000,0.058824,0.000000,0.00,0.000000,0.000000,1,#FF6F61
5,2088,-33.830200,151.233926,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1,#FF6F61
6,2089,-33.831717,151.221577,0.000000,0.041667,0.041667,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.041667,0.041667,0.000000,0.00,0.000000,0.000000,1,#FF6F61
7,2090,-33.829859,151.227430,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0,#ff8000
8,75009,48.883429,2.332437,0.000000,0.000000,0.000000,0.00,0.5,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,2,#ff8080
9,75018,48.885830,2.335438,0.000000,0.083333,0.083333,0.00,0.0,0.062500,0.000000,...,0.000000,0.020833,0.000000,0.000000,0.020833,0.00,0.020833,0.020833,1,#ff0000


In [123]:
#Get the color for each cluster.
city_grouped.loc[city_grouped['Label'] == 0, 'color'] = '#000000'
city_grouped.loc[city_grouped['Label'] == 1, 'color'] = '#ff0000'
city_grouped.loc[city_grouped['Label'] == 2, 'color'] = '#ff8000'
city_grouped.loc[city_grouped['Label'] == 3, 'color'] = '#ff8080'
city_grouped.loc[city_grouped['Label'] == 4, 'color'] = '#FF6F61' 
city_grouped.loc[city_grouped['Label'] == 5, 'color'] = '#6B5B95' 
city_grouped.loc[city_grouped['Label'] == 6, 'color'] = '#92A8D1' 
city_grouped.loc[city_grouped['Label'] == 7, 'color'] = '#B565A7' 
city_grouped.loc[city_grouped['Label'] == 8, 'color'] = '#009B77' 
city_grouped.loc[city_grouped['Label'] == 9, 'color'] = '#DD4124' 
city_grouped.loc[city_grouped['Label'] == 10, 'color'] = '#45B8AC' 
city_grouped.head (20)

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,...,South Indian Restaurant,Street Art,Tennis Court,Thai Restaurant,Theater,Trail,Wine Bar,Wine Shop,Label,color
0,2055,-33.834709,151.206667,0.000000,0.500000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,3,#ff8080
1,2060,-33.837990,151.208102,0.038462,0.000000,0.115385,0.00,0.0,0.038462,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1,#ff0000
2,2061,-33.844796,151.212978,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0,#000000
3,2062,-33.822669,151.211810,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,1,#ff0000
4,2065,-33.826548,151.201621,0.000000,0.000000,0.058824,0.00,0.0,0.000000,0.058824,...,0.058824,0.000000,0.000000,0.058824,0.000000,0.00,0.000000,0.000000,1,#ff0000
5,2088,-33.830200,151.233926,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1,#ff0000
6,2089,-33.831717,151.221577,0.000000,0.041667,0.041667,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.041667,0.041667,0.000000,0.00,0.000000,0.000000,1,#ff0000
7,2090,-33.829859,151.227430,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0,#000000
8,75009,48.883429,2.332437,0.000000,0.000000,0.000000,0.00,0.5,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,2,#ff8000
9,75018,48.885830,2.335438,0.000000,0.083333,0.083333,0.00,0.0,0.062500,0.000000,...,0.000000,0.020833,0.000000,0.000000,0.020833,0.00,0.020833,0.020833,1,#ff0000


In [124]:
MontmatreCluster = city_grouped.loc[city_grouped['postalCode'] == '75018']['Label'].values
MontmatreCluster

array([1], dtype=int32)

Plug the data on a graph.

world_map = folium.Map(location=[48.880507 ,2.336721 ],zoom_start =12)
venues = folium.map.FeatureGroup()

for lat, lng,color in zip(city_grouped.lat, city_grouped.lng,city_grouped.color):
    venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            color=color,
            fill=True,
            fill_color='color',
            fill_opacity=0.6
        )
    )
world_map.add_child(venues)
# display map
world_map

In [125]:
SydneyAreasLikeParis = city_grouped.loc[(city_grouped['postalCode'].astype('int64') <=10000) & (city_grouped['Label'] ==MontmatreCluster[0])  ]
SydneyAreasLikeParis

,postalCode,lat,lng,American Restaurant,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,...,South Indian Restaurant,Street Art,Tennis Court,Thai Restaurant,Theater,Trail,Wine Bar,Wine Shop,Label,color
1,2060,-33.837990,151.208102,0.038462,0.000000,0.115385,0.00,0.0,0.038462,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1,#ff0000
3,2062,-33.822669,151.211810,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.200000,0.0,0.0,0.0,0.0,1,#ff0000
4,2065,-33.826548,151.201621,0.000000,0.000000,0.058824,0.00,0.0,0.000000,0.058824,...,0.058824,0.0,0.000000,0.058824,0.0,0.0,0.0,0.0,1,#ff0000
5,2088,-33.830200,151.233926,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1,#ff0000
6,2089,-33.831717,151.221577,0.000000,0.041667,0.041667,0.00,0.0,0.000000,0.000000,...,0.000000,0.0,0.041667,0.041667,0.0,0.0,0.0,0.0,1,#ff0000


In [132]:
sydney_area_map = folium.Map(location=[ -33.87285279645642,151.2065676235312],zoom_start =12)
area = folium.map.FeatureGroup()

for lat, lng,color in zip(SydneyAreasLikeParis.lat, SydneyAreasLikeParis.lng,SydneyAreasLikeParis.color):
    area.add_child(
        folium.CircleMarker(
            [lat, lng],
            color=color,
            fill=True,
            fill_color='color',
            fill_opacity=0.6,
            
        )
    )
sydney_area_map.add_child(area)
# display map
sydney_area_map

Display the venus for those areas

In [307]:
VenuesSydney= SydneyAreasLikeParis[['postalCode']].merge(df_venues, how='left', left_on='postalCode', right_on='postalCode')
VenuesSydney

,postalCode,name,categories,lat,lng,city
0,2060,St Leonards Park,Park,-33.831122,151.210446,North Sydney
1,2060,Scuro Espresso,Café,-33.831115,151.205855,North Sydney
2,2060,Vienna Sandwiches,Sandwich Place,-33.839046,151.208281,North Sydney
3,2060,The Greens,Bistro,-33.831771,151.211139,North Sydney
4,2060,The Northside Bunker,Café,-33.839241,151.208792,North Sydney
...,...,...,...,...,...,...
71,2089,3 Addictions,Restaurant,-33.831740,151.223910,Neutral Bay
72,2089,The Lab Cafe,Café,-33.831699,151.223535,Neutral Bay
73,2089,Sushi Komaru 鮨 鼓丸,Japanese Restaurant,-33.831557,151.223095,Neutral Bay
74,2089,Anytime Fitness,Gym,-33.830438,151.219725,Neutral Bay


Get color 1 color for each categorie 

In [308]:
#{'beige', 'black', 'blue', 'cadetblue', 'darkblue', 'darkgreen', 'darkpurple', 'darkred', 'gray', 'green', 'lightblue', 'lightgray', 'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'red', 'white'}
countCat = VenuesSydney['categories'].nunique()
List_Categories=VenuesSydney.categories.unique()
print(countCat)
List_Categories=pd.DataFrame(List_Categories,columns=['categories'])
List_Categories

34


,categories
0,Park
1,Café
2,Sandwich Place
3,Bistro
4,Bar
5,Japanese Restaurant
6,Coffee Shop
7,Garden
8,Juice Bar
9,Market


In [309]:
#Get random color for each venue categorie
List_Categories['color']=0
for cat in List_Categories['categories'] :
    r = lambda: random.randint(0,255)
    color =('#%02X%02X%02X' % (r(),r(),r()))
   # colorlist = list(np.random.choice(range(256), size=3)) 
    
    #color = colorlist[0].astype('str') +','+ colorlist[1].astype('str') +',' +colorlist[2].astype('str')
    print (color)
    List_Categories.loc[List_Categories['categories'] == cat, ['color']] = color
List_Categories


#D63FFE
#775A82
#EC31E6
#DF28A0
#2AB13F
#BDC9C0
#3BB83D
#ABCF53
#1C5135
#B503D1
#DB1C37
#FF200A
#27151B
#096DAA
#43F7EC
#79A36A
#E8A5E5
#108E3C
#4DB3A3
#9AB3E3
#BAEA23
#064E2D
#BA8CEB
#6CB538
#6F481F
#2B35BE
#EDA013
#4F5667
#B3B270
#1CF41F
#FA3C1B
#C641DA
#DD9F00
#23031E


,categories,color
0,Park,#D63FFE
1,Café,#775A82
2,Sandwich Place,#EC31E6
3,Bistro,#DF28A0
4,Bar,#2AB13F
5,Japanese Restaurant,#BDC9C0
6,Coffee Shop,#3BB83D
7,Garden,#ABCF53
8,Juice Bar,#1C5135
9,Market,#B503D1


In [310]:
#Merge venues and color mapping

VenuesSydney= VenuesSydney.merge(List_Categories, how='left', left_on='categories', right_on='categories')
VenuesSydney

,postalCode,name,categories,lat,lng,city,color
0,2060,St Leonards Park,Park,-33.831122,151.210446,North Sydney,#D63FFE
1,2060,Scuro Espresso,Café,-33.831115,151.205855,North Sydney,#775A82
2,2060,Vienna Sandwiches,Sandwich Place,-33.839046,151.208281,North Sydney,#EC31E6
3,2060,The Greens,Bistro,-33.831771,151.211139,North Sydney,#DF28A0
4,2060,The Northside Bunker,Café,-33.839241,151.208792,North Sydney,#775A82
...,...,...,...,...,...,...,...
71,2089,3 Addictions,Restaurant,-33.831740,151.223910,Neutral Bay,#C641DA
72,2089,The Lab Cafe,Café,-33.831699,151.223535,Neutral Bay,#775A82
73,2089,Sushi Komaru 鮨 鼓丸,Japanese Restaurant,-33.831557,151.223095,Neutral Bay,#BDC9C0
74,2089,Anytime Fitness,Gym,-33.830438,151.219725,Neutral Bay,#DD9F00


In [316]:
sydney_venues_map = folium.Map(location=[ -33.87285279645642,151.2065676235312],zoom_start =13)
venues = folium.map.FeatureGroup()

for lat, lng,name,color,cat in zip(VenuesSydney.lat, VenuesSydney.lng,VenuesSydney.name,VenuesSydney.color,VenuesSydney.categories):
    venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            tooltip=name +' - '+cat
        )
    )
sydney_venues_map.add_child(venues)
# display map
sydney_venues_map